In [2]:
from IPython.core.display import HTML,display
import pandas as pd

def prettyTable(data_array, row_labels,col_labels):
    """Show an HTML table from a 2d numpy array"""
    df = pd.DataFrame(data_array,index=row_labels,columns=col_labels)
    table_html = df.to_html()
    return HTML(table_html)

In [3]:
import pandas as pd 

smdf = pd.read_csv('nuc.4.4', sep='\s+', comment='#')
smdict = {}
for k,v in smdf.iteritems():
    ssdict = {}
    for i,j in v.iteritems():
        ssdict[i] = j
    smdict[k] = ssdict

In [4]:
import numpy as np 

s1 = 'ATCCGATACTGGCTATA'
s2 = 'ATCTCTGGAGATCTATAG'

nRows = len(s1) + 1
nCols = len(s2) + 1

rowLabels = [label for label in '0' + s1]
colLabels = [label for label in '0' + s2]

scoreMatrix = np.full([nRows, nCols], 0)
traceback = np.full([nRows, nCols], '.')

display(prettyTable(scoreMatrix, rowLabels, colLabels))

,0,A,T,C,T,C,T,G,G,A,G,A,T,C,T,A,T,A,G
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
T,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
T,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
up_arrow = "\u2191"
right_arrow = "\u2192"
down_arrow = "\u2193"
left_arrow = "\u2190"
down_right_arrow = "\u2198"
up_left_arrow = "\u2196"
stop = "\u002e"

gapPenalty = -1
maxScore = -1
maxScoreIndex = (-1,-1)

for i in range(1, nRows):
    for j in range(1, nCols):
        matchScore = smdict[s1[i-1]][s2[j-1]]
        diagonalScore = scoreMatrix[i-1, j-1] + matchScore
        upScore = scoreMatrix[i-1, j] + gapPenalty
        leftScore = scoreMatrix[i, j-1] + gapPenalty

        score = max(0, diagonalScore, upScore, leftScore)
        scoreMatrix[i, j] = score

        if score == 0:
            traceback[i, j] = stop
        elif score == leftScore:
            traceback[i, j] = left_arrow
        elif score == upScore:
            traceback[i, j] = up_arrow
        elif score == diagonalScore:
            traceback[i, j] = up_left_arrow

        if score >= maxScore:
            maxScore = score
            maxScoreIndex = (i, j)

display(prettyTable(scoreMatrix, rowLabels, colLabels))
display(prettyTable(traceback, rowLabels, colLabels))


,0,A,T,C,T,C,T,G,G,A,G,A,T,C,T,A,T,A,G
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,5,4,3,2,1,0,0,0,5,4,5,4,3,2,5,4,5,4
T,0,4,10,9,8,7,6,5,4,4,3,4,10,9,8,7,10,9,8
C,0,3,9,15,14,13,12,11,10,9,8,7,9,15,14,13,12,11,10
C,0,2,8,14,13,19,18,17,16,15,14,13,12,14,13,12,11,10,9
G,0,1,7,13,12,18,17,23,22,21,20,19,18,17,16,15,14,13,15
A,0,5,6,12,11,17,16,22,21,27,26,25,24,23,22,21,20,19,18
T,0,4,10,11,17,16,22,21,20,26,25,24,30,29,28,27,26,25,24
A,0,5,9,10,16,15,21,20,19,25,24,30,29,28,27,33,32,31,30
C,0,4,8,14,15,21,20,19,18,24,23,29,28,34,33,32,31,30,29


,0,A,T,C,T,C,T,G,G,A,G,A,T,C,T,A,T,A,G
0,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
A,.,↖,←,←,←,←,.,.,.,↖,←,↖,←,←,←,↖,←,↖,←
T,.,↑,↖,←,←,←,←,←,←,↑,←,↑,↖,←,←,←,↖,←,←
C,.,↑,↑,↖,←,←,←,←,←,←,←,←,↑,↖,←,←,←,←,←
C,.,↑,↑,↑,←,↖,←,←,←,←,←,←,←,↑,←,←,←,←,←
G,.,↑,↑,↑,←,↑,←,↖,←,←,←,←,←,←,←,←,←,←,↖
A,.,↖,↑,↑,←,↑,←,↑,←,↖,←,←,←,←,←,←,←,←,←
T,.,↑,↖,↑,↖,←,↖,←,←,↑,←,←,↖,←,←,←,←,←,←
A,.,↖,↑,↑,↑,←,↑,←,←,↑,←,↖,←,←,←,↖,←,←,←
C,.,↑,↑,↖,↑,↖,←,←,←,↑,←,↑,←,↖,←,←,←,←,←


In [7]:
# alignment

s1align = ''
s2align = ''
consensus = ''

i, j = maxScoreIndex

while traceback[i, j] != stop:
    s1char = s1[i - 1]
    s2char = s2[j - 1]
    if traceback[i, j] == up_left_arrow:
        s1align = s1char + s1align
        s2align = s2char + s2align
        if s1char == s2char:
            consensus = '|' + consensus
        else:
            consensus = '.' + consensus
        i = i - 1
        j = j - 1
    elif traceback[i, j] == up_arrow:
        s1align = s1char + s1align
        s2align = '-' + s2align
        consensus = ' ' + consensus
        i = i - 1
    elif traceback[i, j] == left_arrow:
        s1align = '-' + s1align
        s2align = s2char + s2align
        consensus = ' ' + consensus
        j = j - 1

print('{0}\n{1}\n{2}'.format(s1align, consensus, s2align))

    

ATCCGATACTGG----CTATA
|||   | ||||    |||||
ATC---T-CTGGAGATCTATA
